# Week 3: Segmenting and Clustering Neighborhoods in Toronto

## Part 3

### 1. Import the necessary libraries

In [8]:
import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2. Import the dataframe from Part 2

In [9]:
df = pd.read_csv("Coordinates_Toronto_df.csv",index_col=[0])
df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 3. Identify only neighborhoods in Toronto

In [10]:
# Filter only rows where 'Borough' contains the word 'Toronto'
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.head()

print(df_toronto.shape) 

(39, 4)


### 4. Create map of Toronto

In [11]:
# Use Geopy library to get latitude & longitude
address = 'Toronto'
geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude,longitude))

# Create map 
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, long, borough, neighborhood in zip(df_toronto['Latitude'], 
                                            df_toronto['Longitude'], 
                                            df_toronto['Borough'], 
                                            df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 5. Explore neighborhoods with FourSquare API 

In [13]:
# Foursquare Credentials
CLIENT_ID = '4EQ0JYQEL2S2UBGDN30BXJEH4JWXTOCSXQCU0T1HUM3C0GFU'
CLIENT_SECRET = 'EJP0FUPBMSNYGWDVNMREWFDLZZB5VZCPXJCSSS4WHRCHQIUG' 
VERSION = '20180605' 

# Function that gets the top venues within a 500 m radius 
def getNearbyVenues(names, latitudes, longitudes, radius = 500, LIMIT = 100):
    venues = [] #Empty list of venues
    
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # API request 
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Append only inofrmation we want for each nearby venue
        venues.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    # Create dataframe that holds all the information of the venues nearby
    df_venues_nearby = pd.DataFrame([item for venues in venues for item in venues])
    df_venues_nearby.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(df_venues_nearby)

df_venues_toronto = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'])
print(df_venues_toronto.shape) # Find the size of the df
print(df_venues_toronto.head(5))
#print('There are {} uniques venue categories.'.format(len(df_venues_toronto['Venue Category'].unique())))

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

### 6. Investigate further the neighborhoods 

In [14]:
# Create one hot encoding
ohe_toronto = pd.get_dummies(df_venues_toronto['Venue Category'], prefix = '', prefix_sep = '')
 
# Add & Move the Neighborhood column 
ohe_toronto['Neighborhood'] = df_venues_toronto['Neighborhood']
fixed_columns = [ohe_toronto.columns[-1]] + list(ohe_toronto.columns[:-1])
ohe_toronto = ohe_toronto[fixed_columns]
ohe_toronto.head()

# Group rows by neighborhood & take the mean of frequency 
df_grouped = ohe_toronto.groupby('Neighborhood').mean().reset_index()
df_grouped.head()

# Function that sorts the values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# Create columns according to number of top venues
columns = ['Neighborhood']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i + 1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i + 1))

# Create dataframe that shows the top 5 venues in each neighborhood
df_venues_sorted = pd.DataFrame(columns = columns)
df_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for i in np.arange(df_grouped.shape[0]):
    df_venues_sorted.iloc[i, 1:] = return_most_common_venues(df_grouped.iloc[i, :], num_top_venues)

df_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Skate Park,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop


### 7. Create map showing neighborhoods clustered

In [19]:
kclusters = 5

df_grouped_cluster = df_grouped.drop(['Neighborhood'], 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(df_grouped_cluster)
kmeans.labels_[0:10]

# Create new dataframe that includes the cluster & top 5 venues found above
df_toronto_merged = df_toronto.copy()
df_toronto_merged['Cluster Labels'] = kmeans.labels_

# Merge the two datasets on Neighborhood
df_toronto_merged = df_toronto_merged.join(df_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
df_toronto_merged.sort_values(['Cluster Labels'], inplace = True)
print(df_toronto_merged.shape)
df_toronto_merged.head() 

# Create map to visualize the clusters
map_toronto_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Create color scheme of colors for the clusters
colors = {}
for cluster in range(kclusters):
    colors[cluster]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size = 3)) #Random color

markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_merged['Latitude'], 
                                  df_toronto_merged['Longitude'], 
                                  df_toronto_merged['Neighborhood'], 
                                  df_toronto_merged['Cluster Labels']):
    label_text = 'Cluster ' + str(cluster) + ' / ' + str(poi)
    label = folium.Popup(label_text, parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        tooltip = label_text,
        radius = 5,
        popup=label,
        color = colors[cluster],
        fill = True,
        fill_color = colors[cluster],
        fill_opacity = 0.7).add_to(map_toronto_clusters)
       
map_toronto_clusters

(39, 10)


### 8. Explore the clusters 

In [23]:
# Explore 1st Cluster
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 0, df_toronto_merged.columns[[1] + 
                                                     list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Postal Code,,,,,,
M4E,The Beaches,Trail,Health Food Store,Pub,Dog Run,Department Store
M5L,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym
M5N,Roselawn,Garden,Music Venue,Women's Store,Deli / Bodega,Electronics Store
M5P,"Forest Hill North & West, Forest Hill Road Park",Jewelry Store,Trail,Sushi Restaurant,Bus Line,Women's Store
M5R,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,BBQ Joint,Pharmacy
M5S,"University of Toronto, Harbord",Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant
M5W,Stn A PO Boxes,Coffee Shop,Café,Restaurant,Italian Restaurant,Seafood Restaurant
M7A,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Café,Bar,Beer Bar
M5X,"First Canadian Place, Underground city",Coffee Shop,Café,Gym,Japanese Restaurant,Restaurant


In [24]:
# Explore 2nd Cluster
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 1, df_toronto_merged.columns[[1] + 
                                                     list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Postal Code,,,,,,
M5H,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Deli / Bodega,Gym


In [25]:
# Explore 3rd Cluster
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 2, df_toronto_merged.columns[[1] + 
                                                     list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Postal Code,,,,,,
M5V,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique
